In [80]:
train = pd.read_csv('train.csv')
train = train.iloc[:, 1:5]
print(len(train))
train = train.drop_duplicates()
print(len(train))
full_duplicates = len(train)
train = handle_labels(train)
garbage = len(train)
print(len(train))

train_sent = train.loc[:,['sentence', 'sentiment'] ]
train_sent = train_sent[train_sent.duplicated()==False]
print(len(train_sent))
duplicates = train_sent[train_sent.duplicated('sentence', keep = False)==True].sort_values(by = 'sentence')
train_sent = train_sent[train_sent.duplicated('sentence', keep = False)==False]
unique_sent = len(train_sent)
print(len(train_sent))
train_cat = train.loc[:,['sentence', 'category_targets']]
train_cat = train_cat[train_cat.duplicated() == False]
unique_cat = len(train_cat)
print(len(train_cat))

19361
12897
9866
6642
5943
8662


In [84]:
result_sent = {'name': ['full_duplicates', 'garbage', 'unique_sent'], 
          'value': [full_duplicates, garbage, unique_sent]}
result_sent = pd.DataFrame(result_sent)
result_sent

,name,value
0,full_duplicates,12897
1,garbage,9866
2,unique_sent,5943


In [86]:
result_cat = {'name': ['full_duplicates', 'garbage', 'unique_cat'], 
          'value': [full_duplicates, garbage, unique_cat]}
result_cat = pd.DataFrame(result_cat)
result_cat

,name,value
0,full_duplicates,12897
1,garbage,9866
2,unique_cat,8662


In [30]:
#!g1.1
def handle_labels(data):
    mapping = {
        'Communication':'0',
        '?':'1',
        ' ':'5',
        'Quality':'2',
        'Price':'3',
        'Safety':'4'
    }
    
    mapping_sent = {
        '+':'0',
        '?':'1',
        '−':'2'
    }
    data['sentiment'] = data['sentiment'].map(mapping_sent)
    
    data = data.copy()
    data['2category'] = data['2category'].fillna(' ')
    data['1category'] = data['1category'].map(mapping)
    data['2category'] = data['2category'].map(mapping)    
    data['category'] =  data['1category'] + '_' + data['2category']

    data['category_targets'] = data['category'].str.split('_')
    data = data.explode('category_targets')
    data['category_targets'] = data['category_targets'].astype(int)
    data = data[data['category_targets']!= 5]
    data = data[data['category_targets']!= 1]

    return data

In [39]:
train = pd.read_csv('train.csv')
train

,Unnamed: 0,sentence,1category,2category,sentiment
0,4754,При этом всегда получал качественные услуги.,Communication,NaN,+
1,4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",?,NaN,−
2,3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.",?,NaN,−
3,11640,Отвратительное отношение к клиентам.,Communication,NaN,−
4,5571,"Всегда в любое время дня и ночи помогут, ответ...",Communication,NaN,+
...,...,...,...,...,...
19356,8004,Никогда и ни в коем случае не открывайте счет ...,Communication,NaN,−
19357,18182,ТИ откровенно забили на качество и развивают с...,Quality,NaN,−
19358,744,"Я считаю, это прорыв и лидерство финансовых ус...",?,NaN,+
19359,6220,"Писал мужчина очень доходчиво, не финансовым я...",Communication,NaN,+


In [78]:
train = pd.read_csv('train.csv')
train = train.iloc[:, 1:5]
df = train.copy()
garbage_train = len(train) - len(handle_labels(train))
df = handle_labels(train)
full_duplicates = len(df) - len(df.drop_duplicates())
df = df.drop_duplicates()
with_many_sent = len(df) - len(df.drop_duplicates(subset='sentence', keep=False))
df = df.drop_duplicates(subset='sentence', keep=False)
df
result = {'name': ['garbage_train', 'full_duplicates', 'with_many_sent', 'unique'], 
          'value': [garbage_train, full_duplicates, with_many_sent, len(df)]}
result = pd.DataFrame(result)
result

,name,value
0,garbage_train,4927
1,full_duplicates,4823
2,with_many_sent,5619
3,unique,3992


In [72]:
df.sort_values(by='sentence')

,sentence,1category,2category,sentiment,category,category_targets
4208,10.12.2020 20:57 Хотелось бы выразить огромну...,2,5,NaN,2_5,2
5368,"!, на что сотрудник банка ответила мне что дан...",0,5,NaN,0_5,0
16788,"!, тем самым оставив меня без средств к сущест...",2,5,NaN,2_5,2
5286,!22.02.2020 заблокировали счет якобы из-за про...,0,5,NaN,0_5,0
8254,!закрывал ИИС в ОФИСЕ - итог отказ - неправиль...,0,5,NaN,0_5,0
...,...,...,...,...,...,...
12772,что под субсидию ПДКП замечательно подходит и ...,2,5,NaN,2_5,2
8988,"это как следствие того, что они совсем не рабо...",0,5,NaN,0_5,0
16563,"я деньги с карты не снимала, какая задолженнос...",4,5,NaN,4_5,4
18103,я звоню в филиалы отп банка их два в городе но...,2,5,NaN,2_5,2


In [73]:
result.sum()

name     garbage_trainfull_duplicateswith_many_sentunique
value                                               19361
dtype: object

In [74]:
import plotly.express as px
import matplotlib.pyplot as plt 

px.pie(result, values='value', names='name')

In [50]:
train = pd.read_csv('train.csv')
train = train.iloc[:, 1:5]
train = handle_labels(train)
train = train.drop_duplicates()


train_sent = train.loc[:,['sentence', 'sentiment'] ]
train_sent
# train_sent = train_sent[train_sent.duplicated()==False]

,sentence,sentiment
0,При этом всегда получал качественные услуги.,0
3,Отвратительное отношение к клиентам.,2
4,"Всегда в любое время дня и ночи помогут, ответ...",0
5,"Все время согласовывалось, всё делалось быстро.",0
6,Абсолютное бездействие и нежелание банка работ...,2
...,...,...
19344,"Ни заявления, ни документа, подтверждающего оп...",2
19349,И в последний момент (так как карта до августа...,2
19351,"Хотелось бы так же прояснить, что до сложившей...",0
19356,Никогда и ни в коем случае не открывайте счет ...,2


In [22]:
pd.read_csv('1sentencenewtest.csv')

,Unnamed: 0,0
0,0,15.03.2022 обратился на горячую линию для закр...
1,1,"Уже который год в ТКБ не решается ""глобальная ..."
2,2,Добрый день
3,3,"Добрый день Сегодня, зайдя в свой личный кабин..."
4,4,"Обслуживаюсь в Тинькофф пару лет, возникла жес..."
...,...,...
944,944,Отвратительный сервис и отношение к клиентам! ...
945,945,28.04.2022 обратилась в банк о возможности пер...
946,946,В начале 2021 года была акция по выплате 8% ке...
947,947,Бездействие банка и некомпетентность сотрудников


In [26]:
test = pd.read_csv('1sentencenewtest.csv').rename(columns={'0': 'sentence'})
test.to_csv('1sentencenewtest_sentence.csv')

In [27]:
test

,Unnamed: 0,sentence
0,0,15.03.2022 обратился на горячую линию для закр...
1,1,"Уже который год в ТКБ не решается ""глобальная ..."
2,2,Добрый день
3,3,"Добрый день Сегодня, зайдя в свой личный кабин..."
4,4,"Обслуживаюсь в Тинькофф пару лет, возникла жес..."
...,...,...
944,944,Отвратительный сервис и отношение к клиентам! ...
945,945,28.04.2022 обратилась в банк о возможности пер...
946,946,В начале 2021 года была акция по выплате 8% ке...
947,947,Бездействие банка и некомпетентность сотрудников


,Unnamed: 0,sentence
0,4036,15.03.2022 обратился на горячую линию для закр...
1,5804,"Уже который год в ТКБ не решается ""глобальная ..."
2,2752,Добрый день. Хочу оставить отзыв о пользовании...
3,1921,"Добрый день Сегодня, зайдя в свой личный кабин..."
4,7374,"Обслуживаюсь в Тинькофф пару лет, возникла жес..."
...,...,...
995,146,Отвратительный сервис и отношение к клиентам! ...
996,2677,28.04.2022 обратилась в банк о возможности пер...
997,4481,В начале 2021 года была акция по выплате 8% ке...
998,4112,Бездействие банка и некомпетентность сотрудник...


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
def handle_labels(data):
    mapping = {
        'Communication':'0',
        '?':'1',
        ' ':'5',
        'Quality':'2',
        'Price':'3',
        'Safety':'4'
    }
    
    mapping_sent = {
        '+':'0',
        '?':'1',
        '−':'2'
    }
    data['sentiment'] = data['sentiment'].map(mapping_sent)
    
    data = data.copy()
    data['2category'] = data['2category'].fillna(' ')
    data['1category'] = data['1category'].map(mapping)
    data['2category'] = data['2category'].map(mapping)    
    data['category'] =  data['1category'] + '_' + data['2category']

    data['category_targets'] = data['category'].str.split('_')
    data = data.explode('category_targets')
    data['category_targets'] = data['category_targets'].astype(int)
    data = data[data['category_targets']!= 5]
    data = data[data['category_targets']!= 1]

    return data

In [4]:
train = pd.read_csv('train.csv')
train = train.iloc[:, 1:5]
train = train.drop_duplicates()
train = handle_labels(train)



In [5]:
train_sent = train.loc[:,['sentence', 'sentiment'] ]
train_sent = train_sent[train_sent.duplicated()==False]
duplicates = train_sent[train_sent.duplicated('sentence', keep = False)==True].sort_values(by = 'sentence')
train_sent = train_sent[train_sent.duplicated('sentence', keep = False)==False]

In [6]:
train_cat = train.loc[:,['sentence', 'category_targets']]
train_cat = train_cat[train_cat.duplicated() == False]

In [7]:
train_cat

,sentence,category_targets
0,При этом всегда получал качественные услуги.,0
3,Отвратительное отношение к клиентам.,0
4,"Всегда в любое время дня и ночи помогут, ответ...",0
5,"Все время согласовывалось, всё делалось быстро.",0
6,Абсолютное бездействие и нежелание банка работ...,2
...,...,...
19344,"Ни заявления, ни документа, подтверждающего оп...",2
19349,И в последний момент (так как карта до августа...,0
19351,"Хотелось бы так же прояснить, что до сложившей...",2
19356,Никогда и ни в коем случае не открывайте счет ...,0


In [252]:
train_sent.to_csv('sentiments.csv')
train_cat.to_csv('cats.csv')